# LLM tool calling

In [ ]:
import warnings
import json
from datetime import datetime
from litellm import completion
from duckduckgo_search import DDGS
from dotenv import load_dotenv

warnings.filterwarnings("ignore", category=UserWarning)
load_dotenv()

True

In [11]:
# Tool 1: DuckDuckGo Search
def duckduckgo_search(query, max_results=5):
    results = []
    with DDGS() as ddgs:
        for r in ddgs.text(query, max_results=max_results):
            results.append(f"{r['title']}: {r['href']}")
    return "\n".join(results)


# Tool 2: Character Counter
def count_character_count(text):
    return f"The input has {len(text)} characters."


# Tool 3: Character Count in Word
def count_character_occurrence(word, character):
    if len(character) != 1:
        return "Error: Please provide a single character."
    count = word.count(character)
    return f"The character '{character}' appears {count} time(s) in '{word}'."


# Tool 4: Current Datetime
def get_current_datetime():
    return f"The current date and time is: {datetime.now().isoformat()}"


# Tool 5: Weather Search (via DuckDuckGo)
def get_weather(place):
    query = f"current weather in {place}"
    return duckduckgo_search(query, max_results=3)

In [ ]:
# Register tools
tool_functions = {}

def register_tool(func):
    tool_functions[func.__name__] = func
    return func


register_tool(duckduckgo_search)
register_tool(count_character_count)
register_tool(count_character_occurrence)
register_tool(get_current_datetime)
register_tool(get_weather)

# Tool definitions
tools = [
    {
        "type": "function",
        "function": {
            "name": "duckduckgo_search",
            "description": "Search the web using DuckDuckGo",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "Search query"},
                },
                "required": ["query"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "count_character_count",
            "description": "Count number of characters in a string",
            "parameters": {
                "type": "object",
                "properties": {
                    "text": {"type": "string", "description": "The string to analyze"},
                },
                "required": ["text"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "count_character_occurrence",
            "description": "Count how many times a character appears in a word",
            "parameters": {
                "type": "object",
                "properties": {
                    "word": {"type": "string", "description": "Target word"},
                    "character": {"type": "string", "description": "Single character"},
                },
                "required": ["word", "character"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_datetime",
            "description": "Get current date and time",
            "parameters": {
                "type": "object",
                "properties": {},
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Search current weather in a place",
            "parameters": {
                "type": "object",
                "properties": {
                    "place": {"type": "string", "description": "Location to check weather"},
                },
                "required": ["place"],
            },
        }
    }
]


In [13]:
def run_agent(question):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": question}
    ]

    for step in range(3):
        print(f"\n--- Step {step} ---")

        response = completion(
            model="gpt-4.1-mini",
            messages=messages,
            tools=tools,
            tool_choice="auto",
        )

        msg = response['choices'][0]['message']

        if msg.get("tool_calls"):
            tool_call = msg["tool_calls"][0]
            function_name = tool_call["function"]["name"]
            arguments = json.loads(tool_call["function"]["arguments"])

            print(f"\n🔧 Tool requested: {function_name}({arguments})")

            if function_name in tool_functions:
                try:
                    result = tool_functions[function_name](**arguments)
                except Exception as e:
                    result = f"Error running tool '{function_name}': {e}"
            else:
                result = f"❌ Unknown tool: {function_name}"

            messages.append(msg)
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call["id"],
                "name": function_name,
                "content": result,
            })

            print(f"\n📘 Tool Result:\n{result}")
        else:
            final_answer = msg["content"]
            print(f"\n✅ Final Answer:\n{final_answer}")
            break


In [14]:
run_agent("what time is it now?")


--- Step 0 ---

🔧 Tool requested: get_current_datetime({})

📘 Tool Result:
The current date and time is: 2025-06-18T21:18:24.815485

--- Step 1 ---

✅ Final Answer:
The current time is 21:18 (9:18 PM).


In [15]:
run_agent("how many r in strawberry?")


--- Step 0 ---

🔧 Tool requested: count_character_occurrence({'word': 'strawberry', 'character': 'r'})

📘 Tool Result:
The character 'r' appears 3 time(s) in 'strawberry'.

--- Step 1 ---

✅ Final Answer:
The letter 'r' appears 3 times in the word "strawberry."


In [16]:
run_agent("what's the weather in Beijing tomorrow?")


--- Step 0 ---

🔧 Tool requested: get_weather({'place': 'Beijing'})

📘 Tool Result:
Beijing, Beijing, China Weather Forecast | AccuWeather: https://www.accuweather.com/en/cn/beijing/101924/weather-forecast/101924
Beijing Weather Forecast Today: https://chinaweather.org/beijing
Beijing, BJ, CN Current Weather - The Weather Network: https://www.theweathernetwork.com/en/city/cn/beijing/beijing-2/current

--- Step 1 ---

🔧 Tool requested: duckduckgo_search({'query': 'Beijing weather forecast tomorrow'})

📘 Tool Result:
Error running tool 'duckduckgo_search': https://lite.duckduckgo.com/lite/ 202 Ratelimit

--- Step 2 ---

✅ Final Answer:
I am currently experiencing some limitations in accessing up-to-date weather forecast information for Beijing tomorrow. However, you can check reliable weather websites like AccuWeather, The Weather Network, or use a weather app for the latest forecast. Would you like me to help with something else?
